In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [4]:
data18 = data.loc[data['Year']== '2018']
data18 = data18.loc[data18['Genre1']== 'Rock']
data18

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
76,77,A Brief Inquiry Into Online Relationships,The 1975,dare to be too much. led by frontman and lyri...,"Dirty Hit,Interscope",8.5,November,29,2018,Rock,NaN,NaN,NaN
80,81,“Ouch”,Nicholas Krgovich,“you learn that so much of the fear and anxiet...,Tin Angel,7.2,November,29,2018,Rock,NaN,NaN,NaN
82,83,Look Now Look Again,Rainer Maria,met as 21-year-olds in a college poetry class...,Polyvinyl,8.3,November,29,2018,Rock,NaN,NaN,NaN
83,84,Anthem of the Peaceful Army,Greta Van Fleet,greta van fleet sound like they did weed exact...,"Lava,Republic",1.6,October,23,2018,Rock,NaN,NaN,NaN
85,86,The Bela Session,Bauhaus,everyone knows “bela lugosi’s dead.” for decad...,Leaving,7.9,December,1,2018,Rock,NaN,NaN,NaN
93,94,Songs of Love and Horror,Will Oldham,"near the end of a recent show in brooklyn, , b...",Drag City,7.1,October,24,2018,Rock,NaN,NaN,NaN
95,96,Bohemian Rhapsody OST,Queen,"have now existed longer as a band , freddie m...",Virgin,4.5,October,25,2018,Rock,NaN,NaN,NaN


In [22]:
data.Genre1.unique()

array(['Pop/R&B', 'Rock', 'Experimental', 'Rap', 'Electronic',
       'Folk/Country', 'Global', 'Metal', 'Jazz'], dtype=object)

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data18['content']))

print(data_words[:1])

[['you', 'know', 'how', 'life', 'goes', 'one', 'day', 'you', 're', 'making', 'songs', 'like', 'selling', 'with', 'vagina', 'bat', 'flying', 'above', 'your', 'band', 'name', 'and', 'years', 'go', 'by', 'and', 'suddenly', 'everyone', 'getting', 'married', 'and', 'having', 'kids', 'after', 'all', 'the', 'actual', 'blood', 'piss', 'and', 'sweat', 'that', 'went', 'into', 'mocking', 'indie', 'culture', 'and', 'convention', 'your', 'friends', 'have', 'somehow', 'become', 'craft', 'brewers', 'and', 'commercial', 'music', 'writers', 'rhode', 'island', 'hardcore', 'misanthropes', 'never', 'seemed', 'the', 'sort', 'of', 'band', 'compatible', 'with', 'the', 'demands', 'of', 'adulthood', 'when', 'they', 'broke', 'up', 'in', 'due', 'to', 'extreme', 'internal', 'duress', 'there', 'was', 'little', 'reason', 'to', 'believe', 'some', 'newfound', 'cachet', 'changing', 'times', 'or', 'popular', 'demand', 'would', 'ever', 'coax', 'their', 'wildly', 'abrasive', 'music', 'back', 'into', 'action', 'they', 'ac

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['you', 'know', 'how', 'life', 'goes', 'one', 'day', 'you', 're', 'making', 'songs', 'like', 'selling', 'with', 'vagina', 'bat', 'flying', 'above', 'your', 'band', 'name', 'and', 'years', 'go', 'by', 'and', 'suddenly', 'everyone', 'getting', 'married', 'and', 'having', 'kids', 'after', 'all', 'the', 'actual', 'blood', 'piss', 'and', 'sweat', 'that', 'went', 'into', 'mocking', 'indie', 'culture', 'and', 'convention', 'your', 'friends', 'have', 'somehow', 'become', 'craft', 'brewers', 'and', 'commercial', 'music', 'writers', 'rhode', 'island', 'hardcore', 'misanthropes', 'never', 'seemed', 'the', 'sort', 'of', 'band', 'compatible', 'with', 'the', 'demands', 'of', 'adulthood', 'when', 'they', 'broke', 'up', 'in', 'due', 'to', 'extreme', 'internal', 'duress', 'there', 'was', 'little', 'reason', 'to', 'believe', 'some', 'newfound', 'cachet', 'changing', 'times', 'or', 'popular', 'demand', 'would', 'ever', 'coax', 'their', 'wildly', 'abrasive', 'music', 'back', 'into', 'action', 'they', 'act

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['know', 'life', 'go', 'day', 'make', 'song', 'sell', 'vagina', 'bat', 'fly', 'band', 'name', 'year', 'go', 'suddenly', 'everyone', 'get', 'married', 'kid', 'actual', 'blood', 'piss', 'sweat', 'go', 'mock', 'indie', 'culture', 'convention', 'friend', 'somehow', 'become', 'craft', 'brewer', 'commercial', 'music', 'writer', 'island', 'hardcore', 'misanthrope', 'never', 'seem', 'sort', 'band', 'compatible', 'demand', 'adulthood', 'break', 'due', 'extreme', 'internal', 'duress', 'little', 'reason', 'believe', 'newfound', 'cachet', 'change', 'time', 'popular', 'demand', 'would', 'ever', 'coax', 'wildly', 'abrasive', 'music', 'back', 'action', 'actually', 'trick', 'reunite', 'anyway', 'organizing', 'principle', 'plan', 'linger', 'contempt', 'humanity', 'say', 'way', 'title', 'fourth', 'album', 'first', 'year', 'really', 'mean', 'get', 'deserve', 'daughter', 'work', 'provide', 'instantaneous', 'absurdist', 'release', 'string', 'minute', 'song', 'pummel', 'flee', 'minute', 'almost', 'long', '

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#filter extreme values in dictionary
id2word.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 2), (16, 1), (17, 1), (18, 2), (19, 1), (20, 2), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1)]]


In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [19]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.023*"hall" + 0.021*"country" + 0.018*"texture" + 0.017*"piece" + '
  '0.014*"indulge" + 0.014*"atmosphere" + 0.014*"instrumental" + 0.013*"peace" '
  '+ 0.011*"obscure" + 0.011*"lurch"'),
 (1,
  '0.001*"protest" + 0.001*"surround" + 0.001*"winter" + 0.001*"warm" + '
  '0.001*"nervous" + 0.001*"scream" + 0.001*"particular" + 0.001*"melt" + '
  '0.001*"lesson" + 0.001*"body"'),
 (2,
  '0.113*"white" + 0.046*"hole" + 0.043*"collect" + 0.039*"male" + '
  '0.030*"streak" + 0.026*"third" + 0.025*"british" + 0.023*"bill" + '
  '0.020*"material" + 0.019*"audience"'),
 (3,
  '0.005*"piano" + 0.004*"noise" + 0.004*"number" + 0.004*"blue" + '
  '0.004*"edge" + 0.004*"chord" + 0.004*"songwriting" + 0.004*"mood" + '
  '0.004*"simple" + 0.004*"bassist"'),
 (4,
  '0.095*"woman" + 0.043*"hardcore" + 0.037*"ep" + 0.036*"protest" + '
  '0.035*"female" + 0.029*"scream" + 0.027*"girl" + 0.025*"shoegaze" + '
  '0.022*"split" + 0.022*"angry"'),
 (5,
  '0.181*"sugar" + 0.001*"amount" + 0.001*"learn

In [20]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.354272075548855

Coherence Score:  0.2651154135845917


In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.056480  0.012687       1        1  57.840591
0     -0.070136  0.010004       2        1  31.603283
9     -0.122856 -0.131158       3        1   2.834225
8      0.001885 -0.206781       4        1   2.369169
1      0.291999 -0.082446       5        1   2.119255
2      0.035413  0.132151       6        1   1.607283
7      0.089951  0.153966       7        1   1.016113
4     -0.094126  0.080443       8        1   0.552853
5     -0.040376  0.017717       9        1   0.045847
6     -0.035275  0.013416      10        1   0.011381, topic_info=     Category        Freq          Term       Total  loglift  logprob
617   Default  152.000000         woman  152.000000  30.0000  30.0000
690   Default   99.000000            ep   99.000000  29.0000  29.0000
407   Default   73.000000           emo   73.000000  28.0000  28.0000
239   Default   86.000000          film   86.000000  27.0000  27.0000
85    Default   90.000000          wait   90.000000  26.0000  26.0000
294   Default   90.000000         piece   90.000000  25.0000  25.0000
874   Default   94.000000          girl   94.000000  24.0000  24.0000
819   Default   68.000000         white   68.000000  23.0000  23.0000
631   Default   75.000000           fun   75.000000  22.0000  22.0000
404   Default   96.000000           die   96.000000  21.0000  21.0000
572   Default   88.000000         label   88.000000  20.0000  20.0000
292   Default  107.000000         piano  107.000000  19.0000  19.0000
883   Default   90.000000        matter   90.000000  18.0000  18.0000
116   Default   87.000000       harmony   87.000000  17.0000  17.0000
673   Default   72.000000         third   72.000000  16.0000  16.0000
68    Default   74.000000        scream   74.000000  15.0000  15.0000
793   Default   66.000000          body   66.000000  14.0000  14.0000
560   Default   64.000000        desire   64.000000  13.0000  13.0000
1311  Default   52.000000           war   52.000000  12.0000  12.0000
520   Default   72.000000          pain   72.000000  11.0000  11.0000
109   Default   97.000000    experience   97.000000  10.0000  10.0000
735   Default   84.000000          mood   84.000000   9.0000   9.0000
1170  Default   65.000000         whole   65.000000   8.0000   8.0000
253   Default   83.000000           guy   83.000000   7.0000   7.0000
119   Default   72.000000          hook   72.000000   6.0000   6.0000
858   Default   84.000000     character   84.000000   5.0000   5.0000
303   Default   87.000000      question   87.000000   4.0000   4.0000
511   Default   73.000000         heavy   73.000000   3.0000   3.0000
611   Default   51.000000          trio   51.000000   2.0000   2.0000
1092  Default   40.000000           duo   40.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
715   Topic10    0.003862        wonder   62.559193  -0.6118  -7.2814
820   Topic10    0.003862           age   91.299675  -0.9898  -7.2814
12    Topic10    0.003862          city   88.056038  -0.9536  -7.2814
500   Topic10    0.003862       embrace   57.432045  -0.5262  -7.2814
72    Topic10    0.003862       similar   66.421509  -0.6717  -7.2814
941   Topic10    0.003862    eventually   31.127430   0.0863  -7.2814
422   Topic10    0.003862          grow   86.349197  -0.9340  -7.2814
572   Topic10    0.003862         label   88.258652  -0.9559  -7.2814
565   Topic10    0.003862        figure   57.671440  -0.5304  -7.2814
520   Topic10    0.003862          pain   72.431160  -0.7583  -7.2814
723   Topic10    0.003862       country  107.085213  -1.1493  -7.2814
294   Topic10    0.003862         piece   90.282547  -0.9786  -7.2814
444   Topic10    0.003862     nightmare   20.181730   0.5196  -7.2814
1203  Topic10    0.003862      basement   32.281204   0.0499  -7.2814
1160  Topic10    0.003862         punch   26.542065   0.2456  -7.2814
309   Topic10    0.